### TORCH.AUTOGRAD 에 대한 간단한 소개
torch.autograd 는 신경망 학습을 지원하는 PyTorch의 자동 미분 엔진입니다. 


이 단원에서는 autograd가 신경망 학습을 어떻게 돕는지에 대한 개념적 이해를 할 수 있습니다.

### 배경(Background)


신경망(NN; Neural Network)은 어떤 입력 데이터에 대해 실행되는 중첩(nested)된 함수들의 모음(collection)입니다. 

이 함수들은 PyTorch에서 Tensor로 저장되는, (가중치(weight)와 편향(bias)로 구성된) 매개변수들로 정의됩니다.

신경망을 학습하는 것은 2단계로 이루어집니다:

순전파(Forward Propagation): 순전파 단계에서, 신경망은 정답을 맞추기 위해 최선의 추측(best guess)을 합니다. 이렇게 추측을 하기 위해서 입력 데이터를 각 함수들에서 실행합니다.

역전파(Backward Propagation): 역전파 단계에서, 신경망은 추측한 값에서 발생한 오류(error)에 비례하여(proportionate) 매개변수들을 적절히 조절(adjust)합니다. 출력(output)로부터 역방향으로 이동하면서 오류에 대한 함수들의 매개변수들의 미분값( 변화도(gradient) )을 수집하고, 경사하강법(gradient descent)을 사용하여 매개변수들을 최적화 합니다. 역전파에 대한 자세한 설명은 [3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)의 비디오 를 참고하세요.

SyntaxError: invalid syntax (1781408537.py, line 1)